### Pt0.5 - imports and (mounted) paths

In [79]:
import pandas as pd
import numpy as np
from glob import glob
from os import path,makedirs
import re

In [27]:
bids_dir = '/Users/henrymj/Documents/SRO/tmp/OAK/data/uh2/aim1/BIDS_scans'
deriv_dir = path.join(bids_dir, 'derivatives')
firstlevel_path = path.join(deriv_dir, '1stlevel')
fsl_input_path = path.join(deriv_dir, 'fsl/inputs')

In [4]:
eventsfiles = glob(path.join(firstlevel_path, '*/*', 'simplified_events_RT-True_*')) #the RT regressor does not affect the calculation of other regressors
eventsfiles.sort()

In [5]:
designfiles = glob(path.join(firstlevel_path, '*/*', 'design_RT-True*')) #confounds should not be affected by presence/absence of RT regressor
designfiles.sort()

## Pt1 - making simple events .tsv files for design matrix

In [ ]:
for ef in eventsfiles:
    subjtask = ef.split('1stlevel/')[1].split('/simplified')[0] #get subj/task
    print(subjtask)
    makedirs(path.join(fsl_input_path, subjtask), exist_ok=True) #make fsl/inputs/subj/task dir
    ev_df = pd.read_csv(ef)
    curr_conditions = ev_df.conditions.unique()
    for cond in curr_conditions:
        tmp = ev_df[ev_df.conditions==cond] #get condition subset
        tmp = tmp.filter(['onsets', 'durations', 'amplitudes']) #drop irrelevant columns
        tmp.to_csv(path.join(fsl_input_path, subjtask, cond+'_events.tsv'), sep='\t', header=False, index=False) #save as simple tsv

## Pt2 - making confound .tsv files for design matrix

In [ ]:
confound_cols = ['trans_*', 'rot_*', 'framewise_displacement', 'a_comp_cor_*', 'rejectTR_*', 'drift_*']

In [23]:
for des in designfiles:
    subjtask = des.split('1stlevel/')[1].split('/design')[0] #get subj/task
    makedirs(path.join(fsl_input_path, subjtask), exist_ok=True) #make fsl/inputs/subj/task dir
    des_df = pd.read_csv(des, index_col=0)
    confound_df = des_df.filter(regex='|'.join(confound_cols))
    confound_df.to_csv(path.join(fsl_input_path, subjtask, 'confounds.tsv'), sep='\t', header=False, index=False) #save as simple tsv    

In [21]:
des_df.columns

Index(['response_time', 'response_time_TD', 'task', 'task_TD', 'trans_x',
       'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z', 'trans_xtd',
       'trans_ytd', 'trans_ztd', 'rot_xtd', 'rot_ytd', 'rot_ztd', 'trans_x_sq',
       'trans_y_sq', 'trans_z_sq', 'rot_x_sq', 'rot_y_sq', 'rot_z_sq',
       'trans_xtd_sq', 'trans_ytd_sq', 'trans_ztd_sq', 'rot_xtd_sq',
       'rot_ytd_sq', 'rot_ztd_sq', 'framewise_displacement', 'a_comp_cor_00',
       'a_comp_cor_01', 'a_comp_cor_02', 'a_comp_cor_03', 'a_comp_cor_04',
       'a_comp_cor_05', 'a_comp_cor_06', 'a_comp_cor_07', 'rejectTR_159',
       'rejectTR_160', 'rejectTR_161', 'rejectTR_175', 'rejectTR_228',
       'rejectTR_230', 'rejectTR_259', 'rejectTR_284', 'rejectTR_507',
       'rejectTR_659', 'rejectTR_668', 'rejectTR_669', 'rejectTR_794',
       'rejectTR_795', 'rejectTR_827', 'rejectTR_845', 'rejectTR_846',
       'rejectTR_847', 'drift_1', 'drift_2', 'drift_3', 'drift_4', 'drift_5',
       'drift_6', 'drift_7', 'drift_8', 'drift_9

## Pt3 - extract brains

this will print out comands like the one below, to be pasted into an sbatch file

`/share/software/user/open/fsl/5.0.10/bin/bet /oak/stanford/groups/russpold/data/uh2/aim1/BIDS_scans/sub-s061/ses-1/anat/sub-s061_ses-1_T1w /oak/stanford/groups/russpold/data/uh2/aim1/BIDS_scans/derivatives/fsl/inputs/s061/brain  -f 0.5 -g 0 -m
`

In [84]:
sbatch_headers='''#!/bin/bash
#SBATCH --job-name=fsl_BET
#SBATCH --output=.out/fsl_BET.out
#SBATCH --error=.err/fsl_BET.err
#SBATCH --time=10:00:00
#SBATCH --mail-type=END
#SBATCH --mail-user=poldracklab@stanford.edu
#SBATCH --cpus-per-task=8
#SBATCH --ntasks-per-node=1
#SBATCH -p russpold
export PYTHONPATH=""
'''

In [85]:
fsl_command = '/share/software/user/open/fsl/5.0.10/bin/bet'
sherlock_bids = '/oak/stanford/groups/russpold/data/uh2/aim1/BIDS_scans'
sherlock_fsl = '/oak/stanford/groups/russpold/data/uh2/aim1/BIDS_scans/derivatives/fsl/inputs'
fsl_flags = '  -f 0.5 -g 0 -m'

In [86]:
anats = glob(path.join(bids_dir, 'sub-*', 'ses-*', 'anat', '*_T1w*'))
anats.sort()

In [94]:
f=open('../fsl/fsl_BET.batch', 'w+')

In [95]:
f.write(sbatch_headers)
f.write('\n')

1

In [96]:
for anat in anats:
    full_sub = anat.split('BIDS_scans/')[1].split('/ses')[0]
    subid = full_sub.split('-')[1]

    sesanat = anat.split(full_sub)[1]
    sesanat = sesanat.strip('/')
    
    sesnums = re.findall(r'\d', sesanat)
    sesnum = sesnums[0]

    t1_file = anat.split('/')[-1].replace('.nii.gz', '')

    full_command = fsl_command+' '+path.join(sherlock_bids, full_sub, sesanat, t1_file)+' '+path.join(sherlock_fsl, subid, 'brain'+sesnum)+fsl_flags+'\n'
#     print(full_command)
    f.write(full_command)
    f.write('\n')

/share/software/user/open/fsl/5.0.10/bin/bet /oak/stanford/groups/russpold/data/uh2/aim1/BIDS_scans/sub-s061/ses-1/anat/sub-s061_ses-1_T1w /oak/stanford/groups/russpold/data/uh2/aim1/BIDS_scans/derivatives/fsl/inputs/s061/brain1  -f 0.5 -g 0 -m

/share/software/user/open/fsl/5.0.10/bin/bet /oak/stanford/groups/russpold/data/uh2/aim1/BIDS_scans/sub-s130/ses-1/anat/sub-s130_ses-1_T1w /oak/stanford/groups/russpold/data/uh2/aim1/BIDS_scans/derivatives/fsl/inputs/s130/brain1  -f 0.5 -g 0 -m

/share/software/user/open/fsl/5.0.10/bin/bet /oak/stanford/groups/russpold/data/uh2/aim1/BIDS_scans/sub-s130/ses-2/anat/sub-s130_ses-2_T1w /oak/stanford/groups/russpold/data/uh2/aim1/BIDS_scans/derivatives/fsl/inputs/s130/brain2  -f 0.5 -g 0 -m

/share/software/user/open/fsl/5.0.10/bin/bet /oak/stanford/groups/russpold/data/uh2/aim1/BIDS_scans/sub-s144/ses-1/anat/sub-s144_ses-1_T1w /oak/stanford/groups/russpold/data/uh2/aim1/BIDS_scans/derivatives/fsl/inputs/s144/brain1  -f 0.5 -g 0 -m

/share/software/

In [97]:
f.close()